# Assignment 6.2



In [21]:
from functools import wraps
import inspect

### Task 1

Write a decorator that ensures a function is only called by users with a specific role. Each function should have an user_type with a string type in kwargs.

In [17]:
def is_admin(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if kwargs.get('user_type') != 'admin':
            raise ValueError("Permission denied")
        return func(*args, **kwargs)
    return wrapper

@is_admin
def show_customer_receipt(user_type: str):
    print("Showing customer receipt...")

try:
    show_customer_receipt(user_type='user')
except ValueError as e:
    print(e)

#show_customer_receipt(user_type='admin')


Permission denied


### Task 2

Write a decorator that wraps a function in a try-except block and prints an error if any type of error has happened.

In [13]:
def catch_errors(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Found 1 error during execution of your function: {type(e).__name__} {e}")
    return wrapper

@catch_errors
def some_function_with_risky_operation(data):
    print(data['key'])

#some_function_with_risky_operation({'foo': 'bar'})

some_function_with_risky_operation({'key': 'bar'})


bar


### Task 3 (optional)

Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations. It should work for all possible functions. Don`t forget to check the return type as well

In [22]:
def check_types(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        annotations = func.__annotations__
        sig = inspect.signature(func)
        bound_args = sig.bind(*args, **kwargs)
        bound_args.apply_defaults()

        for name, value in bound_args.arguments.items():
            if name in annotations:
                expected_type = annotations[name]
                if not isinstance(value, expected_type):
                    raise TypeError

        result = func(*args, **kwargs)

        if 'return' in annotations:
            expected_return_type = annotations['return']
            if not isinstance(result, expected_return_type):
                raise TypeError

        return result
    return wrapper

@check_types
def add(a: int, b: int) -> int:
    return a + b

print(add(1, 2))
#print(add("1", "2"))

3


### Task 4 (optional)

Create a function that caches the result of a function, so that if it is called with the same argument multiple times, it returns the cached result first instead of re-executing the function.

In [16]:
def cache(func):
    cached_results = {}

    @wraps(func)
    def wrapper(*args):
        if args in cached_results:
            print(f"Fetching cached result for {args}")
            return cached_results[args]

        print(f"Calculating result for {args}")
        result = func(*args)
        cached_results[args] = result
        return result

    return wrapper

@cache
def expensive_computation(x):
    return x * x

print(expensive_computation(4))
print(expensive_computation(4))
print(expensive_computation(5))
print(expensive_computation(4))


Calculating result for (4,)
16
Fetching cached result for (4,)
16
Calculating result for (5,)
25
Fetching cached result for (4,)
16


### Task 5 (optional)

Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute